# IBM Data Science Capstone Final Project

### Introduction

As part of my final capstone project for IBM Data Science Professional Certificate in Coursera, I choose to explore neighborhoods for Riyadh city, where i'm living,  using Foursquare location data. Riyadh is a metopletanian area with more than 6 million (macrotrends web site) people living there, the tunisian people who immigrate to Saudia arabia and scpecially to Riyadh are in exponential increase and the need to find and enjoy tunisian cuisine is on the rise. Due to the limit number of tunsian restaurant in Riyadh, the idea to find more suitable place to begin such project comes up.

### Business Problem

The queston to answer is **"Where is the more suitable place to launch such project?"** in Riyadh, Saudia Arbia. I will try to answer this question by using data science, machine learning method (clustering) and Foresquare Api to extract needed data.


### Target Audiance

for any entrepreneur or businessman who wants to find a location to lanch such project

### Data

The data needed for the project is:
- Riyadh location (Latitude and Longitude)
- Neighborhood list in Riyadh city.
- Latitude and Longitude relative to these neihborhoods.
- Venue List of north african restaurant.

#### Datasets

1- List of neighborhood from https://en.wikipedia.org/wiki/Riyadh#City_districts

2- Foresquare Api to extract the venue list for each neighborhood</div>

#### Extracting the data
- scapping Riyadh neighborhood via Wikipedia
- Get latitude and longitude for each neighborhood extracted in previous section using geocoder
- extract venue list of each neighborhood using foresquare Api

In [1]:
#!pip install folium
#!pip install beautifulsoup4
#!pip install lxml
#!pip install geopy

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


from urllib.request import urlopen
import re
from geopy.geocoders import Nominatim

print('Librairies Imported')

Librairies Imported


In [9]:
def replacer(s, newstring, index, nofail=False):
    # raise an error if index is outside of the string
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string")

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + 1:]

def cleanTxt(txt):
    idx_start=txt.find("(")
    idx_end=txt.find(")")
    
    if idx_start <0:
        return txt
    if idx_start <0:
        return txt
    return txt[:idx_start] + txt[idx_end+1:]

fn_repl = lambda a: a.replace("'","")

def getAdressLocation(adress):
    adress=adress + ", Riyadh"
    latitude=0
    longitude=0
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    if location != None:
        latitude = location.latitude
        longitude = location.longitude
    
    return latitude, longitude

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Scrapping Riyadh Neighborhood list

In [3]:
html = urlopen("https://en.wikipedia.org/wiki/Riyadh#City_districts").read()
soup = BeautifulSoup(html)
divparent = soup.find_all('div', attrs={'class':'div-col columns column-width'})[0]
ldistr=[]
for row in divparent.find_all('ul'):
    for data in row.find_all('li'):
        if data.string!=None:
            #print(data.string,"--__--" ,cleanTxt(data.string))
            entry=cleanTxt(fn_repl(data.string))
            ldistr.append(entry)

columns=['District']
df_org=pd.DataFrame(ldistr,columns=columns)
df_org.head()


,District
0,Al-Deerah
1,Mikal
2,Manfuha
3,Manfuha Al-Jadidah
4,Al-Oud


#### Drop duplicates

In [4]:
print("dataset shape before : ",df_org.shape)
# sorting by first name 
df_org.sort_values("District", inplace = True) 
df_org
# dropping ALL duplicte values 
df=df_org.drop_duplicates() 
print("dataset shape after : ",df.shape)


dataset shape before :  (196, 1)
dataset shape after :  (95, 1)


#### Get AL-Riyadh Coordinate

In [5]:
address = 'Riyadh, SA'
geolocator = Nominatim()
location = geolocator.geocode(address)
ar_lat = location.latitude
ar_long = location.longitude
print('Al-Riyadh center longitude={}, latitude={}'.format(ar_lat, ar_long))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


Al-Riyadh center longitude=24.6319692, latitude=46.7150648


### Get Riyadh Neighborhood coordinate
we need to get the coordinate of Riyadh neighborhoods in order to get the list of venues

In [6]:
d=[]
for index, row in df.iterrows():
    address = row['District'] + ", Riyadh"
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    if location != None:
        latitude = location.latitude
        longitude = location.longitude
        d.append({'District': row['District'], 'latitude': latitude, 'longitude':longitude})
        
df_new=pd.DataFrame(d)
df_new.head()
df_new.to_csv('Riyadh_Neib_Loc.csv')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


In [7]:
df_loc=pd.read_csv('Riyadh_Neib_Loc.csv',index_col=0)
df_loc=df_loc[['District','latitude','longitude']]
df_loc.head()

,District,latitude,longitude
0,Ad Difa,24.592784,46.833947
1,Al Iskan,21.400517,39.780900
2,Al Izdihar,24.780321,46.717530
3,Al Mansouriyah,24.625390,46.522381
4,Al-Arid,24.499165,47.000378


In [10]:
CLIENT_ID = '************************'
CLIENT_SECRET = '****************************'
VERSION = '20200201' # Foursquare API version
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 750 # define radius

Riyadh_venues = getNearbyVenues(names=df_loc['District'],latitudes=df_loc['latitude'], longitudes=df_loc['longitude'])
Riyadh_venues.shape

Riyadh_venues.head()
Riyadh_venues.to_csv("Riyadh_venues.csv")


Ad Difa
Al Iskan
Al Izdihar
Al Mansouriyah
Al-Arid
Al-Badia
Al-Deerah 
Al-Fayha
Al-Ghadir
Al-Hair
Al-Hamra
Al-Khozama
Al-Maathar
Al-Maathar Al-Shimali 
Al-Maizliyyah
Al-Malaz
Al-Malga
Al-Manakh
Al-Manar
Al-Margab
Al-Marwah
Al-Masani
Al-Masif
Al-Murabba
Al-Murooj
Al-Nafil
Al-Nahdhah
Al-Nakheel
Al-Naseem 
Al-Olayya
Al-Oud
Al-Qadisiyah
Al-Qouds
Al-Rabwah
Al-Raid
Al-Rayyan
Al-Rimayah
Al-Saadah
Al-Shemaysi
Al-Shifa
Al-Urayja
Al-Urayja 
Al-Urayja Al-Wusta 
Al-Wadi
Al-Yamamah
As-Salam
As-Suwaidi
As-Suwaidi 
Diplomatic Quarter
Eleyshah
Hazm
Hijrat Laban
Hittin
Irqah
Jabrah
Jarir
King Abdullah Financial District
King Fahd District
Manfuha
Manfuha Al-Jadidah 
Mikal
Nemar
Otayyigah
Qortubah 
Salam
Shubra
Sultanah
Syah
Taybah
Tuwaiq
Umm Al-Hamam 


In [17]:

Riyadh_venues=pd.read_csv("Riyadh_venues.csv")
Riyadh_venues.drop(['Unnamed: 0'], axis = 1,inplace=True)
Riyadh_venues.head()


#Number of unique venue categories
print('There are {} uniques categories.'.format(len(Riyadh_venues['Venue Category'].unique())))

#Venue count per Category
print(Riyadh_venues.groupby('Venue Category')['Neighborhood'].count())


There are 166 uniques categories.
Venue Category
ATM                       2
Afghan Restaurant         2
Airport Service           1
American Restaurant      10
Antique Shop              2
Aquarium                  1
Arepa Restaurant          2
Art Gallery               1
Arts & Crafts Store       1
Asian Restaurant          2
Assisted Living           1
Athletics & Sports        3
Australian Restaurant     1
Auto Garage               1
Auto Workshop             2
Bakery                   10
Bank                      1
Beach                     1
Beach Bar                 1
Bookstore                 3
Boutique                  2
Boxing Gym                1
Breakfast Spot           23
Bubble Tea Shop           1
Buffet                    1
Burger Joint             16
Bus Stop                  1
Business Service          1
Café                     27
Candy Store               7
                         ..
Salon / Barbershop        2
Sandwich Place           14
Seafood Restaurant        1

#### Filter Only restaurant category

In [19]:
Riyadh_venues
Riyadh_restau = Riyadh_venues[Riyadh_venues['Venue Category'].str.contains('Restaurant')]
Riyadh_restau.to_csv("Riyadh_restau.csv")

#selected_cat=['Restaurant','Sandwich','Breakfast','Falafel','Burger','Pizza','Shawarma']

Riyadh_restau=pd.read_csv("Riyadh_restau.csv")
Riyadh_restau.drop(['Unnamed: 0'], axis = 1,inplace=True)
Riyadh_restau.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Al Iskan,21.400517,39.78090,مطعم الدومان للكباب الميرو,21.401180,39.780820,Mediterranean Restaurant
1,Al Iskan,21.400517,39.78090,Herfy (هرفي),21.400527,39.780757,Fast Food Restaurant
2,Al Iskan,21.400517,39.78090,شاورمتك,21.400806,39.781047,Arepa Restaurant
3,Al Izdihar,24.780321,46.71753,فطائر تركية,24.779479,46.719074,Turkish Restaurant
4,Al Izdihar,24.780321,46.71753,فلافل ابو عدنان,24.778129,46.720055,Falafel Restaurant


##### Finally our data is ready to be analyzed